In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from copy import deepcopy

df=pd.read_csv('C:/Users/cash/Desktop/Mobilis/mobilis_django/filtered_data.csv')

In [ ]:

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371
    return c * r

def Estimate_Time(start, end,speed_kmh):
    lon1, lat1 = start['Longitude'], start['Latitude']
    lon2, lat2 = end['Longitude'], end['Latitude']
    
    distance_km = haversine(lon1, lat1, lon2, lat2)
    
    
    time_hours = distance_km / speed_kmh
    time_minutes = time_hours * 60
    
    return {
        'distance_km': round(distance_km, 2),
        'estimated_time_minutes': round(time_minutes, 2)
    }


start = {'Longitude': 2.033, 'Latitude': 36.752}
end = {'Longitude': 3.0588, 'Latitude': 36.7538}
print(Estimate_Time(start, end,49))


In [ ]:
def print_pretty_result(result):
    if result['points_visited']<result['Original_points']+1:
        print(f'Visiting All the points In the time limit is unlikely to be optimal. You can try to increase the time limit or average speed.')
    print(f"\n📍 Points Visited: {result['points_visited']}")
    print(f"⏱️ Total Travel Time: {result['total_time_minutes']} minutes")
    print("\n🗺️ Visit Order:")
    for i, point in enumerate(result['visited_points'], start=1):
        name = point.get('name', f"Unnamed ({point['Latitude']}, {point['Longitude']})")
        print(f"  {i}. {name}")

In [ ]:


class PlanOptimizerFull(object):
    def __init__(self, data, time_limit, constraints=None, speed=40, daily_limit=480):
        self.original_data = deepcopy(data)
        self.data = deepcopy(data)
        self.global_time_limit = time_limit
        self.constraints = constraints
        self.speed = speed
        self.daily_limit = daily_limit
        self.start_point = None

    def dfs(self, current, remaining, path, total_time):
        if total_time > self.daily_limit:
            return

        if len(path) > len(self.best_path):
            self.best_path = deepcopy(path)
            self.best_time = total_time

        for i, point in enumerate(remaining):
            travel_time = Estimate_Time(current, point, self.speed)['estimated_time_minutes']
            self.dfs(
                point,
                remaining[:i] + remaining[i+1:],
                path + [point],
                total_time + travel_time
            )

    def optimize(self, start, available_points):
        self.best_path = []
        self.best_time = 0
        self.dfs(start, available_points, [start], 0)
        return {
            'visited_points': self.best_path,
            'total_time_minutes': round(self.best_time, 2),
            'points_visited': len(self.best_path)
        }

    def multi_day_optimize(self, start):
        self.start_point = start
        remaining_points = deepcopy(self.data)
        daily_results = []
        total_time_used = 0

        while remaining_points and total_time_used < self.global_time_limit:
            # Remove already visited points from previous runs
            current_remaining = [
                p for p in remaining_points
                if not (p['Longitude'] == start['Longitude'] and p['Latitude'] == start['Latitude'])
            ]

            day_result = self.optimize(start, current_remaining)
            visited = day_result['visited_points'][1:]  # exclude start point from removal
            daily_time = day_result['total_time_minutes']

            if not visited:
                break  # can't fit any more points

            # Add day result and update
            daily_results.append({
                'day': len(daily_results) + 1,
                'path': day_result['visited_points'],
                'time': daily_time
            })
            total_time_used += daily_time
            remaining_points = [
                p for p in remaining_points
                if p not in visited
            ]

        return daily_results



In [73]:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]


start_point = {'name': "Start", 'Longitude': 2.9, 'Latitude': 36.75}
optimizer = PlanOptimizerFull(data=data_points, time_limit=2000, speed=60, daily_limit=480)
results = optimizer.multi_day_optimize(start=start_point)

for r in results:
    print(f"\n📅 Day {r['day']}:")
    for i, pt in enumerate(r['path']):
        print(f"  {i+1}. {pt['name']}")
    print(f"⏱️ Estimated Time: {round(r['time'], 2)} minutes")


KeyboardInterrupt: 

In [74]:
from collections import deque
from copy import deepcopy

class PlanOptimizerFull_BFS(object):
    def __init__(self, data, time_limit, constraints=None, speed=40, daily_limit=480):
        self.original_data = deepcopy(data)
        self.data = deepcopy(data)
        self.global_time_limit = time_limit
        self.constraints = constraints
        self.speed = speed
        self.daily_limit = daily_limit
        self.start_point = None

    def bfs(self, start, remaining):
        # BFS initialization
        queue = deque([(start, remaining, [start], 0)])  # (current_point, remaining_points, path, time_spent)
        best_path = []
        best_time = 0

        while queue:
            current, remaining_points, path, total_time = queue.popleft()

            # If the total time exceeds the daily limit, discard this path
            if total_time > self.daily_limit:
                continue

            # Update the best path if necessary
            if len(path) > len(best_path):
                best_path = deepcopy(path)
                best_time = total_time

            # Process each remaining point
            for i, point in enumerate(remaining_points):
                travel_time = Estimate_Time(current, point, self.speed)['estimated_time_minutes']
                new_path = path + [point]
                new_remaining = remaining_points[:i] + remaining_points[i+1:]

                # Push the new state into the queue
                queue.append((point, new_remaining, new_path, total_time + travel_time))

        return best_path, best_time

    def optimize(self, start, available_points):
        best_path, best_time = self.bfs(start, available_points)
        return {
            'visited_points': best_path,
            'total_time_minutes': round(best_time, 2),
            'points_visited': len(best_path)
        }

    def multi_day_optimize(self, start):
        self.start_point = start
        remaining_points = deepcopy(self.data)
        daily_results = []
        total_time_used = 0

        while remaining_points and total_time_used < self.global_time_limit:
            # Remove already visited points from previous runs
            current_remaining = [
                p for p in remaining_points
                if not (p['Longitude'] == start['Longitude'] and p['Latitude'] == start['Latitude'])
            ]

            day_result = self.optimize(start, current_remaining)
            visited = day_result['visited_points'][1:]  # exclude start point from removal
            daily_time = day_result['total_time_minutes']

            if not visited:
                break  # can't fit any more points

            # Add day result and update
            daily_results.append({
                'day': len(daily_results) + 1,
                'path': day_result['visited_points'],
                'time': daily_time
            })
            total_time_used += daily_time
            remaining_points = [
                p for p in remaining_points
                if p not in visited
            ]

        return daily_results


# Example usage:




In [ ]:
# Example usage:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]


start_point = {'name':"Start",'Longitude': 2.9, 'Latitude': 36.75}
optimizer = PlanOptimizerFull_BFS(data=data_points, time_limit=2000, speed=60)
result = optimizer.multi_day_optimize(start=start_point)

# Call after optimization
print_pretty_result(result)


In [ ]:
data_points = [
    {'name': "Point A", 'Latitude': 36.75, 'Longitude': 2.9},
    {'name': "Point B", 'Latitude': 36.76, 'Longitude': 2.91},
    {'name': "Point C", 'Latitude': 36.77, 'Longitude': 2.92},
    {'name': "Point D", 'Latitude': 36.78, 'Longitude': 2.93},
    {'name': "Point E", 'Latitude': 36.79, 'Longitude': 2.94},
    {'name': "Point F", 'Latitude': 36.80, 'Longitude': 2.95},
    # Add more points as needed
]

start_point = {'name': "Start", 'Longitude': 2.9, 'Latitude': 36.75}

# Set the time limit in minutes (8 hours = 480 minutes)
optimizer = PlanOptimizerFull(data=data_points, time_limit=480, speed=20, daily_limit=480)

result = optimizer.optimize(start=start_point, available_points=data_points)

# Call after optimization
print("Optimization Results:")
print(f"Path: {[point['name'] for point in result['visited_points']]}")
print(f"Total Time: {result['total_time_minutes']} minutes")
print(f"Points Visited: {result['points_visited']}")

In [7]:
import random

# Define the bounds for the coordinates
latitude_bounds = (36.69, 36.80)
longitude_bounds = (2.85, 3.21)

# Function to generate random points
def generate_random_points(num_points):
    points = []
    for i in range(num_points):
        latitude = round(random.uniform(latitude_bounds[0], latitude_bounds[1]), 4)
        longitude = round(random.uniform(longitude_bounds[0], longitude_bounds[1]), 4)
        points.append({
            'name': f'Point {chr(65 + i)}',  # Letters A, B, C, ...
            'longitude': longitude,
            'latitude': latitude
        })
    return points

# Generate 40 random points
data_points = generate_random_points(40)



In [ ]:
import math
import pulp
import numpy as np

# Helper: Haversine formula to compute distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in km
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    return R * 2 * math.asin(math.sqrt(a))

# Plan route for a single day (or time chunk)
def plan_route(points, speed_kmph, time_limit_minutes, visited_points, visit_cost_minutes=15):
    N = len(points)

    # Create subset of unvisited points, ensuring start point is always included
    unvisited_points = [point for point in points if point['name'] not in visited_points and point['name'] != 'Start']
    unvisited_points.insert(0, points[0])  # Always start at the same location
    N_unvisited = len(unvisited_points)

    # Compute travel time matrix (in minutes)
    travel_time = np.zeros((N_unvisited, N_unvisited))
    for i in range(N_unvisited):
        for j in range(N_unvisited):
            if i != j:
                dist = haversine(unvisited_points[i]['longitude'], unvisited_points[i]['latitude'],
                                 unvisited_points[j]['longitude'], unvisited_points[j]['latitude'])
                time = (dist / speed_kmph) * 60
                travel_time[i][j] = time

    # Define integer programming model
    model = pulp.LpProblem("Orienteering", pulp.LpMaximize)

    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(N_unvisited) for j in range(N_unvisited) if i != j), cat="Binary")
    u = pulp.LpVariable.dicts("u", (i for i in range(N_unvisited)), lowBound=0, upBound=N_unvisited - 1, cat="Continuous")

    # Objective: maximize number of visits (excluding return to start)
    model += pulp.lpSum(x[i, j] for i in range(N_unvisited) for j in range(N_unvisited) if i != j and j != 0)

    # Constraints
    model += pulp.lpSum(x[0, j] for j in range(1, N_unvisited)) == 1
    model += pulp.lpSum(x[j, 0] for j in range(1, N_unvisited)) == 0

    for i in range(1, N_unvisited):
        model += pulp.lpSum(x[j, i] for j in range(N_unvisited) if j != i) <= 1
        model += pulp.lpSum(x[i, j] for j in range(N_unvisited) if j != i) <= 1

    model += pulp.lpSum(x[i, j] * (travel_time[i][j] + visit_cost_minutes)
                        for i in range(N_unvisited) for j in range(N_unvisited) if i != j) <= time_limit_minutes

    # Subtour elimination (MTZ)
    for i in range(1, N_unvisited):
        for j in range(1, N_unvisited):
            if i != j:
                model += u[i] - u[j] + (N_unvisited - 1) * x[i, j] <= N_unvisited - 2

    model.solve()

    # Extract solution
    solution_edges = [(i, j) for i in range(N_unvisited) for j in range(N_unvisited)
                      if i != j and pulp.value(x[i, j]) == 1]

    # Build ordered route
    solution_order = [unvisited_points[0]['name']]
    current = 0
    while True:
        next_nodes = [j for i, j in solution_edges if i == current]
        if not next_nodes:
            break
        current = next_nodes[0]
        solution_order.append(unvisited_points[current]['name'])

    estimated_time = sum(travel_time[i][j] + visit_cost_minutes for i, j in solution_edges)

    return solution_order, solution_edges, unvisited_points, estimated_time

# Multi-day planning until all points are visited
def plan_multiple_days(points, daily_limit_minutes, speed_kmph):
    visited_points = []
    all_routes = []
    all_edges = []
    all_estimates = []

    while len(visited_points) < len(points) - 1:  # Exclude start point
        route, edges, unvisited_points, estimated_time = plan_route(
            points, speed_kmph=speed_kmph, time_limit_minutes=daily_limit_minutes, visited_points=visited_points
        )

        if len(route) <= 1:  # No progress
            break

        all_routes.append(route)
        all_edges.append(edges)
        all_estimates.append(estimated_time)
        visited_points.extend(route[1:])  # Skip start point

    return all_routes, all_edges, all_estimates




In [ ]:
start_point = {'name':"Start",'longitude': 2.9, 'latitude': 36.75}

result = plan_multiple_days([start_point]+data_points, daily_limit_minutes=480, speed_kmph=50)

# Call after optimization

NameError: name 'print_pretty_result' is not defined

In [16]:
print(len(result[0]))


8
